In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/BPM')
root = os.getcwd()
!pwd

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM


# Download data

In [ ]:
os.chdir("./raw_data")
os.system("wget -c http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz")
os.system("gzip -d meta_Electronics.json.gz")

os.system("wget -c http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Office_Products.json.gz")
os.system("gzip -d meta_Office_Products.json.gz")

In [ ]:
import gzip
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [ ]:
#Creating an empty list
data = []

#Decoding the gzip file
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

#Defining f as the file that will contain json data
f = open("output_strict.json", 'w')

#Defining linebreak as '\n' and writing one at the end of each line
for l in parse(root + "/raw_data/meta_Electronics.json.gz"):
  f.write(l + '\n')

for l in parse(root + "/raw_data/meta_Office_Products.json.gz"):
  f.write(l + '\n')

#Appending each json element to the empty 'data' list
with open('output_strict.json', 'r') as f:
    for l in tqdm(f):
        data.append(json.loads(l))

#Reading 'data' as a pandas dataframe
full = pd.DataFrame(data)
full.to_csv(root+'/cleaned_data/filtered.csv')
full.shape

<ipython-input-70-857b8c696d5f>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for l in tqdm(f):


0it [00:00, ?it/s]

(633034, 9)

In [ ]:
print(data[0])

{'asin': '0132793040', 'imUrl': 'http://ecx.images-amazon.com/images/I/31JIPhp%2BGIL.jpg', 'description': 'The Kelby Training DVD Mastering Blend Modes in Adobe Photoshop CS5 with Corey Barker is a useful tool for becoming familiar with the use of blend modes in Adobe Photoshop. For those who are serious about mastering all that Photoshop has to offer, mastering blend modes is just as important as mastering layers.In this DVD tutorial, seasoned expert Corey Barker explores the function of blend modes in a variety of scenarios such as image restoration, sharpening, adjustments, special effects and much more. Since every project scenario is different, Corey encourages you to experiment with these blend modes by giving you the skills and confidence you need.', 'categories': [['Electronics', 'Computers & Accessories', 'Cables & Accessories', 'Monitor Accessories']], 'title': 'Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker'}


In [ ]:
print(full.iloc[705])

asin                                                  9983901269
imUrl          http://ecx.images-amazon.com/images/I/31eSNcWV...
description                                                  NaN
categories     [[Electronics, Computers & Accessories, Cables...
title          Professional Kingston MicroSD 2GB (2 Gigabyte)...
price                                                        NaN
salesRank                 {'Cell Phones & Accessories': 2071844}
related                                                      NaN
brand                                                        NaN
Name: 705, dtype: object


#Preprocessing

Delete row with NaN

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(root+"/cleaned_data/filtered.csv")
# Drop unuseful fields
df = df.drop(['price', 'salesRank', 'related'], axis=1)

# Drop rows with NaN value in selected columns
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna(subset=['title', 'description', 'categories', 'imUrl', 'brand'])

df = df.reset_index(drop=True)

df.to_csv(root+"/cleaned_data/filtered.csv", index=False)
df.head()

,Unnamed: 0,asin,imUrl,description,categories,title,brand
0,6,0594012015,http://ecx.images-amazon.com/images/I/41kQsHT8...,The Barnes &amp;amp; Noble BNADPN31 Power Kit ...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble NOOK Power Kit in Carbon BN...,Barnes &amp; Noble
1,13,0594549507,http://ecx.images-amazon.com/images/I/01eX9rjx...,"Slim, smart, and lightweight, this ingenious c...","[['Electronics', 'eBook Readers & Accessories'...",Groovy Protective Stand Cover: Custom Designed...,Barnes &amp; Noble
2,20,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,Barnes &amp; Noble
3,22,0594033934,http://ecx.images-amazon.com/images/I/31Vl-YJ5...,The Barnes &amp;amp; Noble B505360439 Wren Quo...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B505360439 Wren Quote Cover,Barnes &amp; Noble
4,25,059400232X,http://ecx.images-amazon.com/images/I/31zdIqAs...,The Barnes &amp;amp; Noble B211-4023 Johnson Q...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B211-4023 Johnson Quote Cover,Barnes &amp; Noble


In [ ]:
df = pd.read_csv(root+'/cleaned_data/filtered.csv')
print(df.shape)

(180002, 7)


Check correctness

In [ ]:
nan_values = df[df['imUrl'].isna()]
print (nan_values)

Empty DataFrame
Columns: [asin, imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['description'].isna()]
print (nan_values)

Empty DataFrame
Columns: [Unnamed: 0, asin, imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['title'].isna()]
print (nan_values)

Empty DataFrame
Columns: [Unnamed: 0, asin, imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['brand'].isna()]
print (nan_values)

Empty DataFrame
Columns: [Unnamed: 0, asin, imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['description'] == '']
print (nan_values)

Empty DataFrame
Columns: [Unnamed: 0, asin, imUrl, description, categories, title, brand]
Index: []


Creating data soup

In [ ]:
df['ensemble'] = df['title'] + ' ' + \
                 df['categories'].astype(str) + ' ' + \
                 df['brand']

#print first record
df['ensemble'].iloc[100]

"Professional Kingston MicroSDHC 16GB (16 Gigabyte) Card for Samsung Galaxy Proclaim Phone with custom formatting and Standard SD Adapter. (SDHC Class 4 Certified) [['Electronics', 'Computers & Accessories', 'Cables & Accessories', 'Memory Cards', 'Micro SD Cards']] Kingston"

## Cleaning

In [ ]:
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
forbidden_words = set(stopwords.words('english'))

#cleaning function
def text_cleaning(text):
  text = re.sub(r'amp','',text)
  text = re.sub(r'\s+',' ',re.sub('[^A-Za-z0-9]',' ', text.strip().lower())).strip()
  text = [word for word in text.split() if word not in forbidden_words]
  return ' '.join(text)

#apply text cleaning
df['ensemble'] = df['ensemble'].apply(lambda text: text_cleaning(text))

df.to_csv(root+"/cleaned_data/preprocessed.csv")

#print example row
df['ensemble'].iloc[100]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'professional kingston microsdhc 16gb 16 gigabyte card samsung galaxy proclaim phone custom formatting standard sd adapter sdhc class 4 certified electronics computers accessories cables accessories memory cards micro sd cards kingston'

Range of ensemble field

In [ ]:
import pandas as pd
import nltk

# Calcola il numero di parole per ogni riga nella colonna 'text'
df['word_count'] = df['ensemble'].apply(lambda n: len(n.split()))

# Calcola il numero minimo e massimo di parole
min_words = df['word_count'].min()
max_words = df['word_count'].max()

# Stampa i risultati
print("Numero minimo di parole:", min_words)
print("Numero massimo di parole:", max_words)

Numero minimo di parole: 5
Numero massimo di parole: 356


In [ ]:
min_row_index = df['word_count'].idxmin()
print(df.loc[min_row_index])

Unnamed: 0                                                 43521
asin                                                  B000A4BFYQ
imUrl          http://ecx.images-amazon.com/images/I/31aDgVDn...
description         AT&T; 24079 Headset 24079 Headsets & Earsets
categories          [['Electronics', 'Computers & Accessories']]
title                                              24079 Headset
brand                                                   AT&amp;T
ensemble         24079 headset electronics computers accessories
word_count                                                     5
Name: 10115, dtype: object


In [ ]:
print(df.loc[5000])

In [ ]:
from requests import get
from io import BytesIO
from PIL import Image
import pandas as pd
import os

df = pd.read_csv(root+"/cleaned_data/preprocessed.csv")
d = df["imUrl"].iloc[5000]
print(d)
image_raw = get(df["imUrl"].iloc[0])
image = Image.open(BytesIO(image_raw.content))
width, height = image.size

print(width, height)

# Fine-tuning

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import sys
!{sys.executable} -m pip install diffusers transformers accelerate scipy safetensors torch pandas

In [ ]:
import sys
print(sys.version)


3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]


In [ ]:
#pip install diffusion==1.4.0


Dreambooth

In [ ]:
!pip install git
!git clone https://github.com/adobe-research/custom-diffusion.git
!cd custom-diffusion
!git clone https://github.com/CompVis/stable-diffusion.git
!cd stable-diffusion
!conda env create -f environment.yaml
!conda activate ldm
!pip install clip-retrieval tqdm

In [ ]:
!wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt

In [ ]:
## download dataset
# !wget https://www.cs.cmu.edu/~custom-diffusion/assets/data.zip
# !unzip data.zip

## run training (30 GB on 2 GPUs)
!bash scripts/finetune_real.sh "cat" data/cat real_reg/samples_cat  cat finetune_addtoken.yaml .

## save updated model weights
!python src/get_deltas.py --path logs/. --newtoken 1

## sample
!python sample.py --prompt "<new1> cat playing with a ball" --delta_ckpt logs/./checkpoints/delta_epoch\=000004.ckpt --ckpt .

bash: scripts/finetune_real.sh: No such file or directory
python3: can't open file '/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM/src/get_deltas.py': [Errno 2] No such file or directory
python3: can't open file '/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM/sample.py': [Errno 2] No such file or directory


In [ ]:
!bash scripts/finetune_gen.sh "cat" data/cat gen_reg/samples_cat  cat finetune_addtoken.yaml .

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `bash scripts/finetune_gen.sh "cat" data/cat gen_reg/samples_cat  cat finetune_addtoken.yaml <pretrained-model-path>'


# Testing

In [ ]:
!git clone https://github.com/huggingface/diffusers

fatal: destination path 'diffusers' already exists and is not an empty directory.


In [ ]:
import os
root = "/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM/"
os.chdir(root+'/diffusers')
!pwd

/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM/diffusers


In [ ]:
!pip install -e .

In [ ]:
!pip install accelerate torchvision transformers>=4.25.1 ftfy tensorboard Jinja2 clip-retrieval

In [ ]:
!accelerate config default

2023-05-23 14:12:52.452463: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!pip install clip-retrieval
!python retrieve.py --class_prompt cat --class_data_dir real_reg/samples_cat --num_class_images 200

In [ ]:
!export MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export OUTPUT_DIR="."
!export INSTANCE_DIR="./data/cat"
!accelerate launch train_custom_diffusion.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --class_data_dir=./real_reg/samples_cat/ \
  --with_prior_preservation --real_prior --prior_loss_weight=1.0 \
  --class_prompt="cat" --num_class_images=200 \
  --instance_prompt="photo of a <new1> cat"  \
  --resolution=512  \
  --train_batch_size=2  \
  --learning_rate=1e-5  \
  --lr_warmup_steps=0 \
  --max_train_steps=250 \
  --scale_lr --hflip  \
  --modifier_token "<new1>" \
  --validation_prompt="<new1> cat sitting in a bucket"

#Create dataset

##Unescape

In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM'

In [ ]:
# df = pd.read_csv("cleaned_data/preprocessed.csv")
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# df.to_csv("cleaned_data/preprocessed.csv", index=False)
# df

In [ ]:
df = pd.read_csv("cleaned_data/preprocessed.csv")
df

,asin,imUrl,description,categories,title,brand,ensemble
0,0594012015,http://ecx.images-amazon.com/images/I/41kQsHT8...,The Barnes &amp;amp; Noble BNADPN31 Power Kit ...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble NOOK Power Kit in Carbon BN...,Barnes &amp; Noble,barnes noble nook power kit carbon bnadpn31 el...
1,0594549507,http://ecx.images-amazon.com/images/I/01eX9rjx...,"Slim, smart, and lightweight, this ingenious c...","[['Electronics', 'eBook Readers & Accessories'...",Groovy Protective Stand Cover: Custom Designed...,Barnes &amp; Noble,groovy protective stand cover custom designed ...
2,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,Barnes &amp; Noble,barnes noble ov hb adp universal power kit ele...
3,0594033934,http://ecx.images-amazon.com/images/I/31Vl-YJ5...,The Barnes &amp;amp; Noble B505360439 Wren Quo...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B505360439 Wren Quote Cover,Barnes &amp; Noble,barnes noble b505360439 wren quote cover elect...
4,059400232X,http://ecx.images-amazon.com/images/I/31zdIqAs...,The Barnes &amp;amp; Noble B211-4023 Johnson Q...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B211-4023 Johnson Quote Cover,Barnes &amp; Noble,barnes noble b211 4023 johnson quote cover ele...
...,...,...,...,...,...,...,...
105466,B00AJHCSV8,http://ecx.images-amazon.com/images/I/41SWVX33...,A High Speed HDMI Cable with Red Mere from Mon...,"[['Electronics', 'Accessories & Supplies', 'Au...",Monoprice 40-Feet Ultra Slim Series High Perfo...,Monoprice,monoprice 40 feet ultra slim series high perfo...
105467,B00AJHCPYI,http://ecx.images-amazon.com/images/I/41QzWfcP...,A High Speed HDMI Cable with Red Mere from Mon...,"[['Electronics', 'Accessories & Supplies', 'Au...",Monoprice 15-Feet Ultra Slim Series High Perfo...,Monoprice,monoprice 15 feet ultra slim series high perfo...
105468,B00AJHCPIO,http://ecx.images-amazon.com/images/I/41lzWza0...,USB Sync Cable for all iPad iPod and iPhone wi...,"[['Electronics', 'Computers & Accessories', 'C...",Monoprice 109414 3-Feet SlimFit 30-Pin USB Syn...,Monoprice,monoprice 109414 3 feet slimfit 30 pin usb syn...
105469,B00AJHD3RG,http://ecx.images-amazon.com/images/I/31qs0vup...,Monoprice brings you high quality bulk rolls o...,"[['Electronics', 'Computers & Accessories', 'C...",Monoprice 1000-Feet 24AWG Cat5e 350MHz UTP Sol...,Monoprice,monoprice 1000 feet 24awg cat5e 350mhz utp sol...


In [ ]:
from bs4 import BeautifulSoup
from html import unescape

# Applica la funzione BeautifulSoup a ciascun elemento della colonna 'title'
df['title'] = df['title'].apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)

# Stampa il DataFrame aggiornato
print(df['title'])

<ipython-input-24-8bd08f748877>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['title'] = df['title'].apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)


0          Barnes & Noble NOOK Power Kit in Carbon BNADPN31
1         Groovy Protective Stand Cover: Custom Designed...
2              Barnes & Noble OV/HB-ADP Universal Power Kit
3                Barnes & Noble B505360439 Wren Quote Cover
4              Barnes & Noble B211-4023 Johnson Quote Cover
                                ...                        
105466    Monoprice 40-Feet Ultra Slim Series High Perfo...
105467    Monoprice 15-Feet Ultra Slim Series High Perfo...
105468    Monoprice 109414 3-Feet SlimFit 30-Pin USB Syn...
105469    Monoprice 1000-Feet 24AWG Cat5e 350MHz UTP Sol...
105470    Monoprice 109442 10-Feet Premier Series 1/4-In...
Name: title, Length: 105471, dtype: object


In [ ]:
from bs4 import BeautifulSoup
from html import unescape

# Applica la funzione BeautifulSoup a ciascun elemento della colonna 'title'
df['description'] = df['description'].apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)

# Stampa il DataFrame aggiornato
print(df['description'])

<ipython-input-25-9966f1b7c6d2>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['description'] = df['description'].apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)
<ipython-input-25-9966f1b7c6d2>:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  df['description'] = df['description'].apply(lambda x: BeautifulSoup(unescape(x), 'lxml').text)


0         The Barnes & Noble BNADPN31 Power Kit in Carbo...
1         Slim, smart, and lightweight, this ingenious c...
2         Power up your device with this Barnes & Noble ...
3         The Barnes & Noble B505360439 Wren Quote Cover...
4         The Barnes & Noble B211-4023 Johnson Quote Cov...
                                ...                        
105466    A High Speed HDMI Cable with Red Mere from Mon...
105467    A High Speed HDMI Cable with Red Mere from Mon...
105468    USB Sync Cable for all iPad iPod and iPhone wi...
105469    Monoprice brings you high quality bulk rolls o...
105470    Connect your musical instruments and effects p...
Name: description, Length: 105471, dtype: object


In [ ]:
import pandas as pd

# Crea il nuovo DataFrame df2 con le colonne 'imUrl' e 'title' di df
df2 = df[['imUrl', 'title', 'description']].copy()
df2 = df2.dropna(subset=['title', 'description'])

# Stampa il nuovo DataFrame df2
print(df2)

df2.to_csv("cleaned_data/url_title_descr.csv", index=False)

                                                    imUrl  \
0       http://ecx.images-amazon.com/images/I/41kQsHT8...   
1       http://ecx.images-amazon.com/images/I/01eX9rjx...   
2       http://ecx.images-amazon.com/images/I/41K7ymN5...   
3       http://ecx.images-amazon.com/images/I/31Vl-YJ5...   
4       http://ecx.images-amazon.com/images/I/31zdIqAs...   
...                                                   ...   
105466  http://ecx.images-amazon.com/images/I/41SWVX33...   
105467  http://ecx.images-amazon.com/images/I/41QzWfcP...   
105468  http://ecx.images-amazon.com/images/I/41lzWza0...   
105469  http://ecx.images-amazon.com/images/I/31qs0vup...   
105470  http://ecx.images-amazon.com/images/I/31WpW55m...   

                                                    title  \
0        Barnes & Noble NOOK Power Kit in Carbon BNADPN31   
1       Groovy Protective Stand Cover: Custom Designed...   
2            Barnes & Noble OV/HB-ADP Universal Power Kit   
3              Barnes &

##Check if url exists

In [2]:
!pip install datasets
!pip install "tqdm>=4.9.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import pandas as pd

In [5]:
import requests
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

df = pd.read_csv("cleaned_data/url_title_descr.csv")

# Funzione per verificare se un URL è valido
def check_url_validity(url):
    try:
        response = requests.head(url)
        return 1 if response.status_code == 200 else 0
    except requests.exceptions.RequestException:
        return 0

# Creazione della colonna "valid" con tqdm
df['valid'] = df['imUrl'].progress_apply(check_url_validity)

df.to_csv("cleaned_data/valid_url.csv", index=False)

# Visualizzazione del DataFrame risultante
df

100%|██████████| 105471/105471 [2:33:13<00:00, 11.47it/s]


,imUrl,title,description,valid
0,http://ecx.images-amazon.com/images/I/41kQsHT8...,Barnes & Noble NOOK Power Kit in Carbon BNADPN31,The Barnes & Noble BNADPN31 Power Kit in Carbo...,1
1,http://ecx.images-amazon.com/images/I/01eX9rjx...,Groovy Protective Stand Cover: Custom Designed...,"Slim, smart, and lightweight, this ingenious c...",1
2,http://ecx.images-amazon.com/images/I/41K7ymN5...,Barnes & Noble OV/HB-ADP Universal Power Kit,Power up your device with this Barnes & Noble ...,1
3,http://ecx.images-amazon.com/images/I/31Vl-YJ5...,Barnes & Noble B505360439 Wren Quote Cover,The Barnes & Noble B505360439 Wren Quote Cover...,1
4,http://ecx.images-amazon.com/images/I/31zdIqAs...,Barnes & Noble B211-4023 Johnson Quote Cover,The Barnes & Noble B211-4023 Johnson Quote Cov...,1
...,...,...,...,...
105466,http://ecx.images-amazon.com/images/I/41SWVX33...,Monoprice 40-Feet Ultra Slim Series High Perfo...,A High Speed HDMI Cable with Red Mere from Mon...,1
105467,http://ecx.images-amazon.com/images/I/41QzWfcP...,Monoprice 15-Feet Ultra Slim Series High Perfo...,A High Speed HDMI Cable with Red Mere from Mon...,1
105468,http://ecx.images-amazon.com/images/I/41lzWza0...,Monoprice 109414 3-Feet SlimFit 30-Pin USB Syn...,USB Sync Cable for all iPad iPod and iPhone wi...,1
105469,http://ecx.images-amazon.com/images/I/31qs0vup...,Monoprice 1000-Feet 24AWG Cat5e 350MHz UTP Sol...,Monoprice brings you high quality bulk rolls o...,1


In [ ]:
def to_pillow(examples):
    urls = examples['imUrl']
    valid_flags = []

    for url in urls:
        try:
            image = Image.open(requests.get(url, stream=True).raw)
            valid_flags.append(1)
        except:
            valid_flags.append(0)

    examples['valid'] = valid_flags

    return examples

dataset = load_dataset("csv", "cleaned_data/url_title_descr.csv")
dataset = dataset.map(to_pillow, batched=True)

In [ ]:
dataset

#Categories preprocessing

In [ ]:
df = pd.read_csv("amazon/valid_url.csv")
df

In [ ]:
def clean_cat(text):
    text = ' and '.join(text)


df['categories'] = df['categories'][0].apply(lambda text: text_cleaning(text))

#Fashion dataset

In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM'

In [ ]:
data = pd.read_csv('fashion/styles.csv', on_bad_lines='skip')
data

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


In [ ]:
import pandas as pd
import numpy as np

# Drop rows with NaN value in selected columns
data = data.replace(r'^\s*$', np.nan, regex=True)
data = data.dropna(subset=['year', 'productDisplayName'])
data = data.astype({'year':'int'})

data = data.reset_index(drop=True)

# data.to_csv(root+"/fashion/filtered.csv", index=False)
data

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44411,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013,Casual,Gas Men Caddy Casual Shoe
44412,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011,Casual,Lotto Men's Soccer Track Flip Flop
44413,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual,Puma Men Graphic Stellar Blue Tshirt
44414,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017,Casual,Rasasi Women Blue Lady Perfume


In [ ]:
data['keywords'] = data['gender'] + ',' + data['masterCategory'] + ',' + data['subCategory'] + ',' + data['articleType'] \
                              + ',' + data['baseColour'] + ',' + data['season'] + ',' + data['year'].astype(str)  + ',' + data['usage']
data

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,keywords
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,"Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011..."
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,"Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,..."
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,"Women,Accessories,Watches,Watches,Silver,Winte..."
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,"Men,Apparel,Bottomwear,Track Pants,Black,Fall,..."
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,"Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,C..."
...,...,...,...,...,...,...,...,...,...,...,...
44411,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013,Casual,Gas Men Caddy Casual Shoe,"Men,Footwear,Shoes,Casual Shoes,White,Summer,2..."
44412,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011,Casual,Lotto Men's Soccer Track Flip Flop,"Men,Footwear,Flip Flops,Flip Flops,Red,Summer,..."
44413,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual,Puma Men Graphic Stellar Blue Tshirt,"Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual"
44414,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017,Casual,Rasasi Women Blue Lady Perfume,"Women,Personal Care,Fragrance,Perfume and Body..."


In [ ]:
f = data[['keywords', 'productDisplayName']].copy()
f = f.dropna(subset=['keywords', 'productDisplayName'])
f.rename(columns={"productDisplayName": "title"}, inplace=True)
f.to_csv("fashion/keywords_title.csv", index=False)
f

<ipython-input-39-19d88b552f63>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f.rename(columns={"productDisplayName": "title"}, inplace=True)


,keywords,title
0,"Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011...",Turtle Check Men Navy Blue Shirt
1,"Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,...",Peter England Men Party Blue Jeans
2,"Women,Accessories,Watches,Watches,Silver,Winte...",Titan Women Silver Watch
3,"Men,Apparel,Bottomwear,Track Pants,Black,Fall,...",Manchester United Men Solid Black Track Pants
4,"Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,C...",Puma Men Grey T-shirt
...,...,...
44411,"Men,Footwear,Shoes,Casual Shoes,White,Summer,2...",Gas Men Caddy Casual Shoe
44412,"Men,Footwear,Flip Flops,Flip Flops,Red,Summer,...",Lotto Men's Soccer Track Flip Flop
44413,"Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual",Puma Men Graphic Stellar Blue Tshirt
44414,"Women,Personal Care,Fragrance,Perfume and Body...",Rasasi Women Blue Lady Perfume
